In [ ]:
# gradio 설치
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/3

In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import io
from PIL import Image

# 데이터셋 로드 및 전처리
wine = load_wine()
X = wine.data
y = wine.target

# 타겟을 원-핫 인코딩으로 변환
y = tf.keras.utils.to_categorical(y)

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화 (평균 0, 표준편차 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델 정의 함수
def create_model():
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    return model

# 모델 학습 및 평가 함수
def train_model(optimizer_name, learning_rate):
    optimizers = {
        'SGD': tf.keras.optimizers.SGD(learning_rate=learning_rate),
        'Adam': tf.keras.optimizers.Adam(learning_rate=learning_rate),
        'RMSprop': tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    }

    optimizer = optimizers[optimizer_name]
    model = create_model()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # 모델 평가
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # 학습 및 검증 손실 시각화
    plt.figure(figsize=(12, 8))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss ({optimizer_name})')
    plt.legend()

    # 이미지로 변환
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    loss_image = Image.open(buf)
    plt.close()

    # 학습 및 검증 정확도 시각화
    plt.figure(figsize=(12, 8))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Training and Validation Accuracy ({optimizer_name})')
    plt.legend()

    # 이미지로 변환
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    accuracy_image = Image.open(buf)
    plt.close()

    return f"Optimizer: {optimizer_name}, Learning Rate: {learning_rate}, Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}", loss_image, accuracy_image

In [ ]:
# 그라디오 인터페이스 정의
with gr.Blocks() as demo:

    performance = gr.Textbox(label="Model Performance")

    with gr.Row():
        loss_image = gr.Image(type="numpy", label="Training and Validation Loss")
        accuracy_image = gr.Image(type="numpy", label="Training and Validation Accuracy")

    with gr.Row():
        optimizer = gr.Dropdown(['SGD', 'Adam', 'RMSprop'], label="Optimizer", value="Adam")
        learning_rate = gr.Slider(1e-8, 1e-2, label="Learning Rate", value=1e-3)

    btn = gr.Button("Train Model")
    btn.click(train_model, inputs=[optimizer, learning_rate], outputs=[performance, loss_image, accuracy_image])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://05b8bdc9abe96f20c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 실습
- 신용카드 부정사용 탐지 모델을 학습
- https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download